# Download GEDI imagery
#### GEDI04_A - Level 4A Footprint Above Ground Biomass
#### [ORNL DAAC](https://daac.ornl.gov/cgi-bin/dsviewer.pl?ds_id=2056)
#### [ORNL GEDI Tutorial](https://github.com/ornldaac/gedi_tutorials)

In [141]:
bbox = [29.269398, -1.554519, 29.482104, -1.375895]
start_date = dt.datetime(2021, 9, 1) 
end_date = dt.datetime(2022, 6, 1)

In [110]:
import datetime as dt
import geopandas as gpd
import pandas as pd
import rasterio
import requests
from shapely.geometry import MultiPolygon, Polygon, box


# DOI: Digital Object Identifiers
doi = '10.3334/ORNLDAAC/2056' # GEDI L4A DOI 

# CMS: Common Metadata Repository
cmr_url = 'https://cmr.earthdata.nasa.gov/search' 
search_url = f'{cmr_url}/collections.json?doi={doi}'

res = requests.get(search_url)

# Unique NASA-given concept ID for the GEDI 4A dataset, used to retrieve relevant files (or granules) for GEDI L4A
concept_id = requests.get(search_url).json()['feed']['entry'][0]['id']
print(concept_id)

C2237824918-ORNL_CLOUD


In [111]:
# CMR formatted start and end times
date_format = '%Y-%m-%dT%H:%M:%SZ'
date_string = start_date.strftime(date_format) + ',' + end_date.strftime(date_format)

bbox_string = ','.join(map(str, bbox))

granules_list = []
page_size, page_num = 2000, 1

while True:    
    cmr_params = {
        "collection_concept_id": concept_id, 
        "page_size": page_size,
        "page_num": page_num,
        "temporal": date_string,
        "bounding_box[]": bbox_string
    }
    
    granules_url = f'{cmr_url}/granules.json'

    granules_res = requests.get(granules_url, params=cmr_params)
    granules = granules_res.json()['feed']['entry']
        
    if granules:
        for g in granules:

            granule_url = ''
            granule_polygon = ''            
            granule_size = float(g['granule_size'])
            
            if 'polygons' in g:
                polygons= g['polygons']
                multipolygons = []
                for poly in polygons:
                    i = iter(poly[0].split(" "))
                    latlng = list(map(" ".join, zip(i, i)))
                    multipolygons.append(Polygon([[float(p.split(" ")[1]), float(p.split(" ")[0])] for p in latlng]))
                    
                granule_polygon = MultiPolygon(multipolygons)
            
            for links in g['links']:
                if 'title' in links and links['title'].startswith('Download') \
                and links['title'].endswith('.h5'):
                    granule_url = links['href']
                    
            granules_list.append([granule_url, granule_size, granule_polygon])
               
        page_num += 1
        
    else: 
        break

In [112]:
granules_list.append(['bound', 0, box(bbox[0], bbox[1], bbox[2], bbox[3])]) # for plotting
gedi_df = pd.DataFrame(granules_list, columns=["granule_url", "granule_size", "granule_polygon"])
gedi_df = gedi_df[gedi_df['granule_polygon'] != '']

print("Total granules found:", len(gedi_df.index)-1)
print("Total file size (MB): %0.2f" % gedi_df['granule_size'].sum())

Total granules found: 8
Total file size (MB): 1399.12


In [ ]:
gdf = gpd.GeoDataFrame(gedi_df, geometry=gedi_df.granule_polygon)
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
base = world.plot(color='white', edgecolor='black', figsize=(10, 10))

# last row contains the bounding box (Red)
ax = gdf[-1:].plot(ax=base, color='white', edgecolor='red', alpha=1.0)

# all but the last row contains granule bounding geometry (Green)
ax= gdf[:-1].plot(ax=base, color='green', edgecolor='green', alpha=0.4)

minx, miny, maxx, maxy = gdf[-1:].geometry.total_bounds
ax.set_xlim(minx-1, maxx+1)
ax.set_ylim(miny-1, maxy+1)

In [139]:
gedi_granules = gedi_df[:-1].drop_duplicates(subset=['granule_url'])
gedi_granules.to_csv('data/gedi/granules.txt', columns = ['granule_url'], index=False, header = False)

## Setup EarthData Authorization
#### 1. Go to https://urs.earthdata.nasa.gov/ and approve all the ORNL applications (not sure which one matters)
#### 2. Setup credentials on container
- #### cd ~; touch .netrc
- #### echo "machine urs.earthdata.nasa.gov login uid_goes_here password password_goes_here" > .netrc
- #### chmod 0600 .netrc  

In [166]:
%%bash

wget -P data/gedi/ -nv -q --load-cookies ~/.urs_cookies --save-cookies ~/.urs_cookies --auth-no-challenge=on --keep-session-cookies --content-disposition -nc -i data/gedi/granules.txt

In [169]:
import glob
import os

# removes query string from downloaded H5 files
h5_files = glob.glob('./data/gedi/*.h5[?]*')
for file in h5_files:
    os.rename(file, file.split('.h5')[0] + '.h5')

# Download HLS Sentinel-2 imagery
### TODO: complete me!